Welcome to module 2.2! In the last module, we investigated how to use `scikit-learn` to build a simple Naive Bayes model to perform sentiment analysis. In this notebook we will look behind the scenes: we will build our own Naive Bayes model from scratch. We will reuse some of the code we wrote in the previous notebook - but this time, we'll write the maths from scratch.

## ❓ Pre-module quiz

Why is Naive Bayes "naive"?

A. Because it's the most basic, i.e. "naive" classifier we can build

B. Because it "naively" assumes that the probabilities of features (i.e., in our case, words) are independent of each other

C. Because the guy who invented it tought it was a cool name

B. Because it "naively" assumes that the probabilities of features (i.e., in our case, words) are dependent of each other

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>The correct answer is B - Naive Bayes assumes that the probability of finding a certain word is independent from the probability of finding another word. So, for example, in the domain of movie reviews, it assumes that the probability of finding the words `Indiana` and `Jones` are not correlated, even if in practice we know that this is not the case.</p>

</details> 


# Python for Computational Linguists 1.2: Breaking down Naive Bayes

## Introduction

### Recap of last notebook

### Looking back at the theory of Naive Bayes

## Naive Bayes: a simple implementation

From the Post Lecture exercises: 

•  (From J&M-3) Given the following short movie reviews, each
labeled with a genre, either comedy or action:
1. fun, couple, love, love comedy
2. fast, furious, shoot action
3. couple, fly, fast, fun, fun comedy
4. furious, shoot, shoot, fun action
5. fly, fast, shoot, love action

And a new document D: fast, couple, shoot, fly. Compute the
most likely class for D.

### Computing the probabilities

### The training function

### Adding 1-smoothing

## Re-classify the *Thumbs up?* dataset

### Recap of loader functions

### Applying our model

### Improving the results

### Differences between ours and `scikit-learn`

## Wrapping up